<h1 style="color: #007bff;"><center>Impact of COVID-19 & Overdoses on Pharmaceutical Stocks and ETFs</center></h1>
<ul>
<li>Group Number: 21</li>
<li>Student: Karston Kuciemba</li>
<li>UT EID: ktk654</li>
<li>Student: Ananya Kaalva</li>
<li>UT EID: ak46562</li>
<li>Student: Carolyn Pyun</li>
<li>UT EID: cgp755</li>
<li>Date:  3/16/2024</li>
</ul>

In [1]:
# import needed libraries
import numpy as np
import pandas as pd
import math
import altair
import matplotlib.pyplot as plt
import seaborn as sns

<h3 style="padding-left: 30px; color: #007bff;">Read in dataframes for visualization</h3>

In [6]:
# covid dfs
tickers = ['LLY', 'NVO', 'JNJ', 'MRK', 'ABBV', 'MRNA',
           'NVS', 'AZN', 'PFE', 'AMGN', 'PPH', 'IHE', 'PJP']

covid_dfs= {}

for symbol in tickers:
    # for each symbol, load the pkl file and store it in the dictionary as a df
    covid_dfs[f"{symbol}_df-covid"] = pd.read_csv(f"../pharma-data/merged-dfs/{symbol}_df-covid.csv", parse_dates=True)

for symbol in tickers:
    covid_dfs[f"{symbol}_df-covid"] = covid_dfs[f"{symbol}_df-covid"].rename(columns={"Unnamed: 0": "Date"})

In [8]:
# overdose dfs
tickers = ['LLY', 'NVO', 'JNJ', 'MRK', 'ABBV', 'MRNA',
           'NVS', 'AZN', 'PFE', 'AMGN', 'PPH', 'IHE', 'PJP']

overdose_dfs= {}

for symbol in tickers:
    # for each symbol, load the pkl file and store it in the dictionary as a df
    overdose_dfs[f"{symbol}_df-overdose"] = pd.read_csv(f"../pharma-data/merged-dfs/{symbol}_df-overdose.csv", parse_dates=True)

for symbol in tickers:
    overdose_dfs[f"{symbol}_df-overdose"] = overdose_dfs[f"{symbol}_df-overdose"].rename(columns={"Unnamed: 0": "Date"})

# <center style="text-align: center; color: #007bff;">Visualization 1</center>

<h3><center style="color: #DE3163;"><strong>Goal of Visual:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visualization Tools:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Argument Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Present in a bulleted form your argument (add "li" items for each bullet)
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visual Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe the visual that goes with this argument
    </li>
    <li style="font-size: 16px;">
        Describe the interactivity
    </li>
    <li style="font-size: 16px;">
        Describe the layout, choice of colors, annotations (if any), and legends (if any)
    </li>
</ul>

# <center style="text-align: center; color: #007bff;">Visualization 2</center>

<h3><center style="color: #DE3163;"><strong>Goal of Visual:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visualization Tools:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Argument Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Present in a bulleted form your argument (add "li" items for each bullet)
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visual Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe the visual that goes with this argument
    </li>
    <li style="font-size: 16px;">
        Describe the interactivity
    </li>
    <li style="font-size: 16px;">
        Describe the layout, choice of colors, annotations (if any), and legends (if any)
    </li>
</ul>

# <center style="text-align: center; color: #007bff;">Visualization 3</center>

<h3><center style="color: #DE3163;"><strong>Goal of Visual:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visualization Tools:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Argument Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Present in a bulleted form your argument (add "li" items for each bullet)
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visual Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe the visual that goes with this argument
    </li>
    <li style="font-size: 16px;">
        Describe the interactivity
    </li>
    <li style="font-size: 16px;">
        Describe the layout, choice of colors, annotations (if any), and legends (if any)
    </li>
</ul>

# <center style="text-align: center; color: #007bff;">Visualization 4</center>

<h3><center style="color: #DE3163;"><strong>Goal of Visual:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visualization Tools:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Argument Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Present in a bulleted form your argument (add "li" items for each bullet)
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visual Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe the visual that goes with this argument
    </li>
    <li style="font-size: 16px;">
        Describe the interactivity
    </li>
    <li style="font-size: 16px;">
        Describe the layout, choice of colors, annotations (if any), and legends (if any)
    </li>
</ul>

# <center style="text-align: center; color: #007bff;">Visualization 5</center>

<h3><center style="color: #DE3163;"><strong>Goal of Visual:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visualization Tools:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe here
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Argument Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Present in a bulleted form your argument (add "li" items for each bullet)
    </li>
</ul>

<h3><center style="color: #DE3163;"><strong>Visual Layout:</strong></center></h3>
<ul>
    <li style="font-size: 16px;">
        Describe the visual that goes with this argument
    </li>
    <li style="font-size: 16px;">
        Describe the interactivity
    </li>
    <li style="font-size: 16px;">
        Describe the layout, choice of colors, annotations (if any), and legends (if any)
    </li>
</ul>